In [1]:
%load_ext autoreload
%autoreload 2
import nest_asyncio

nest_asyncio.apply()

In [2]:

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from pocketflow_a2a.a2a_common.client import A2AClient
from pocketflow_a2a.a2a_common.types import (
    AgentCard,
    A2AClientJSONError,
    A2AClientHTTPError,
)


client = A2AClient(url="http://localhost:10003")


agent_card = await client.get_agent_card()

ConnectError: All connection attempts failed

In [4]:
print(agent_card.name)
for skill in agent_card.skills:
    print(skill.name)
    print(skill.description)
    print(skill.inputModes)
    print(skill.outputModes)
    print(skill.examples)
    print(skill.tags)
    print(skill.id)
    print(skill.name)

PocketFlow Research Agent (A2A Wrapped)
Web Research and Answering
Answers questions using web search results when necessary.
['text', 'text/plain']
['text', 'text/plain']
['Who won the Nobel Prize in Physics 2024?', 'What is quantum computing?', 'Summarize the latest news about AI.']
['research', 'qa', 'web search']
web_research_qa
Web Research and Answering


# Get available agents

In [25]:
from pocketflow import AsyncFlow, AsyncNode
from typing import TypedDict
import asyncio
from pocketflow_a2a.a2a_client.prompt_templates import available_agents_template


class Shared(TypedDict):
    a2a_clients: list[A2AClient]
    agents_list: list[AgentCard]
    available_agents_prompt: str


class GetAvailableAgents(AsyncNode):
    async def prep_async(self, shared):
        a2a_clients = shared["a2a_clients"]
        return a2a_clients

    async def exec_async(self, a2a_clients: list[A2AClient]):
        return await asyncio.gather(
            *[a2a_client.get_agent_card() for a2a_client in a2a_clients]
        )

    async def exec_fallback_async(self, prep_res, exc):
        raise exc

    async def post_async(
        self,
        shared,
        prep_res,
        exec_res: list[AgentCard | A2AClientJSONError | A2AClientHTTPError],
    ) -> str:
        agents_list = [res for res in exec_res if isinstance(res, AgentCard)]
        if not agents_list:
            shared["agents_list"] = []
            shared["available_agents_prompt"] = None
        else:
            shared["agents_list"] = agents_list
            shared["available_agents_prompt"] = available_agents_template.render(
                agents=[agent.model_dump() for agent in agents_list]
            )
        return "agent_selector"


get_available_agents_node = GetAvailableAgents()

flow = AsyncFlow(start=get_available_agents_node)
shared = {"a2a_clients": [A2AClient(url="http://localhost:10003")]}
await flow.run_async(shared)
print(shared["available_agents_prompt"])

These are agents who can you ask for consult:

Available agents:

Agent number 1:
Name: PocketFlow Research Agent (A2A Wrapped)
Description: A simple research agent based on PocketFlow, made accessible via A2A.
Skills: 
    - name: Web Research and Answering
    - description: Answers questions using web search results when necessary.
    - example: ['Who won the Nobel Prize in Physics 2024?', 'What is quantum computing?', 'Summarize the latest news about AI.']

--------------------------------


You may select one of them to help answer by return the array of selected agent number or if you find no one could help you, you may return empty array.
Your answer will be passed directly to JSON so You must return your selected agent(s) with the following JSON fiedls
[
    {
        "selected_agent: <int, the number of agent that you choose to answer this question>,
        "question_to_answer": <str, the question to ask the agent>
    },
    {/*...other agent you can select as much as you w

In [27]:
from pocketflow_a2a.a2a_common.utils.toolbox import call_llm
from exam.prompt_templates import agent_selector_template

import json
import re

def parse_json_string(s: str) -> dict:
    """
    Parse a string containing JSON (optionally wrapped in markdown code block) into a Python dictionary.
    """
    # Remove leading/trailing whitespace
    s = s.strip()
    # Remove markdown code block if present
    # Handles ```json ... ``` or ``` ... ```
    code_block_pattern = r"^```(?:json)?\s*([\s\S]*?)\s*```$"
    match = re.match(code_block_pattern, s, re.IGNORECASE)
    if match:
        s = match.group(1).strip()
    # Now parse as JSON
    return json.loads(s)


class AgentSelector(AsyncNode):
    
    async def prep_async(self, shared):
        return (shared["question"], shared["available_agents_prompt"])

    async def exec_async(self, inputs):
        question, available_agents_prompt = inputs
        prompt = agent_selector_template.render(
            question=question, available_agents_prompt=available_agents_prompt
        )
        print(prompt)
        return call_llm(prompt)

    async def post_async(self, shared, prep_res, exec_res):
        print(exec_res)
        shared["selected_agent"] = parse_json_string(exec_res)


get_available_agents_node = GetAvailableAgents()
agent_selector_node = AgentSelector()

get_available_agents_node - "agent_selector" >> agent_selector_node

flow = AsyncFlow(start=get_available_agents_node)

shared = {
    "question": "How is Thai stock market this week?",
    "a2a_clients": [A2AClient(url="http://localhost:10003")]
}
await flow.run_async(shared)

print(shared["selected_agent"])


You are a user's question screener. Your duty is to select the agents that proper to answer user's question.

The question: How is Thai stock market this week?

These are agents who can you ask for consult:

Available agents:

Agent number 1:
Name: PocketFlow Research Agent (A2A Wrapped)
Description: A simple research agent based on PocketFlow, made accessible via A2A.
Skills: 
    - name: Web Research and Answering
    - description: Answers questions using web search results when necessary.
    - example: ['Who won the Nobel Prize in Physics 2024?', 'What is quantum computing?', 'Summarize the latest news about AI.']

--------------------------------


You may select one of them to help answer by return the array of selected agent number or if you find no one could help you, you may return empty array.
Your answer will be passed directly to JSON so You must return your selected agent(s) with the following JSON fiedls
[
    {
        "selected_agent: <int, the number of agent that you

In [34]:
print(shared["selected_agent"][0]["selected_agent"] - 1)

0


In [35]:
shared["agents_list"][shared["selected_agent"][0]["selected_agent"] - 1]

AgentCard(name='PocketFlow Research Agent (A2A Wrapped)', description='A simple research agent based on PocketFlow, made accessible via A2A.', url='http://localhost:10003/', provider=None, version='0.1.0-a2a', documentationUrl=None, capabilities=AgentCapabilities(streaming=False, pushNotifications=False, stateTransitionHistory=False), authentication=None, defaultInputModes=['text', 'text/plain'], defaultOutputModes=['text', 'text/plain'], skills=[AgentSkill(id='web_research_qa', name='Web Research and Answering', description='Answers questions using web search results when necessary.', tags=['research', 'qa', 'web search'], examples=['Who won the Nobel Prize in Physics 2024?', 'What is quantum computing?', 'Summarize the latest news about AI.'], inputModes=['text', 'text/plain'], outputModes=['text', 'text/plain'])])

In [ ]:
import json

'```json\n[]\n```\n'

In [ ]:
print(
    agent_selector_template.render(
        question="How is Thai stock market this week?",
        available_agents_prompt=shared["available_agents_prompt"],
    )
)

You are a user's question screener. Your duty is to select the agents that proper to answer user's question.

The question: How is Thai stock market this week?

These are agents who can you ask for consult:

Available agents:

Agent number 1:
Name: PocketFlow Research Agent (A2A Wrapped)
Description: A simple research agent based on PocketFlow, made accessible via A2A.
--------------------------------


You may select one of them to help answer by return the array of selected agent number or if you find no one could help you, you may return empty array.
Your answer will be passed directly to JSON so You must return your selected agent(s) with the following JSON fiedls
[
    {
        "selected_agent: <int, the number of agent that you choose to answer this question>,
        "question_to_answer": <str, the question to ask the agent>
    },
    {/*...other agent you can select as much as you want*/}
]

Remember: Your answer will be passed directly to JSON parsing function, therefore ret

In [11]:
shared["available_agents_prompt"]

'These are agents who can you ask for consult:\n\nAvailable agents:\n\nAgent number 1:\nName: PocketFlow Research Agent (A2A Wrapped)\nDescription: A simple research agent based on PocketFlow, made accessible via A2A.\n--------------------------------\n\n\nYou may select one of them to help answer by return the array of selected agent number or if you find no one could help you, you may return empty array.\nYour answer will be passed directly to JSON so You must return your selected agent(s) with the following JSON fiedls\n[\n    {\n        "selected_agent: <int, the number of agent that you choose to answer this question>,\n        "question_to_answer": <str, the question to ask the agent>\n    },\n    {/*...other agent you can select as much as you want*/}\n]\n\nRemember: Your answer will be passed directly to JSON parsing function, therefore return only the array of json and not include PREFIX, SUFFIX or Prologue'

In [35]:
from jinja2 import Template

# Main template string
main_template = """
Grocery List:
{% for item in items %}
- {% include 'line_template' %}
{% endfor %}
"""

# Sub-template string
line_template = "{{ item.name }} ({{ item.qty }})"

# Create a Jinja2 environment and manually load sub-templates
from jinja2 import DictLoader, Environment

# Setup with in-memory templates
env = Environment(
    loader=DictLoader({"main_template": main_template, "line_template": line_template})
)

# Load and render
template = env.get_template("main_template")

# Sample data
items = [
    {"name": "Milk", "qty": "2 liters"},
    {"name": "Eggs", "qty": "12 pcs"},
    {"name": "Bread", "qty": "1 loaf"},
]

# Render
output = template.render(items=items)
print(output)



Grocery List:

- Milk (2 liters)

- Eggs (12 pcs)

- Bread (1 loaf)

